In [16]:
# In case you want to start all over again, hit the "Run" button on this cell.
%reset -f

In [17]:
# Local Data Point Density-basierte Analyse der Genauigkeit von räumlichen Vorhersagemodellen
# Author: Kieran Galbraith
# Date: 2024-06-09
# Description: Jupyter Notebook to extract reflection data using Google Earth Engine and export it to a CSV file
import geemap
import ee
import time

# Initialize the Earth Engine library
ee.Initialize(project='you-project')

print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [18]:
# Define the area of interest (AOI) as a rectangle
# here: Fijis, source: http://bboxfinder.com/
aoi = ee.Geometry.Rectangle([176.638184, -19.694314, 181.944580, -15.368950])

# or for RLP:
#aoi = ee.Geometry.Polygon([
#    [
#        [6.042480, 49.009051],  
#        [6.042480, 51.062113],  
#        [8.635254, 51.062113],  
#        [8.635254, 49.009051],  
#        [6.042480, 49.009051]   
#    ]
#])

print(aoi.getInfo())


{'type': 'Polygon', 'coordinates': [[[176.638184, -19.694314], [181.94458, -19.694314], [181.94458, -15.36895], [176.638184, -15.36895], [176.638184, -19.694314]]]}


In [19]:
# Function to mask clouds using the Sentinel-2 Scene Classification Layer (SCL) band
def cloudMask(image):
    scl = image.select('SCL')
    mask = scl.eq(3).Or(scl.gte(7).And(scl.lte(11)))
    return image.updateMask(mask.Not())

In [20]:
# Retrieve the Sentinel-2 image collection for the specified date range and filter by cloud percentage
image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2023-05-01', '2023-10-31')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .filterBounds(aoi)\
    .map(cloudMask)\
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])\
    .median()

In [21]:
# Retrieve another Sentinel-2 image collection for a different date range and filter by cloud percentage
# to fill the gaps in the first image
image2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(aoi)\
    .filterDate('2016-01-01', '2022-12-31')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1.0))\
    .map(cloudMask)\
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])\
    .median()\
    .clip(aoi)

In [22]:
combined_img = ee.ImageCollection([image2, image]).mosaic()

In [23]:
# Calculate indices
ndvi = combined_img.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndwi = combined_img.normalizedDifference(['B3', 'B8']).rename('NDWI')
mndwi = combined_img.normalizedDifference(['B3', 'B11']).rename('MNDWI')
ndbi = combined_img.normalizedDifference(['B11', 'B8']).rename('NDBI')
gcvi = combined_img.expression('B8 / B3 - 1', {
    'B8': combined_img.select('B8'),
    'B3': combined_img.select('B3')
}).rename('GCVI')
evi = combined_img.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': combined_img.select('B8'),
        'RED': combined_img.select('B4'),
        'BLUE': combined_img.select('B2')
    }).rename('EVI')

In [ ]:
combined_img = combined_img.addBands([ndvi, ndwi, mndwi, ndbi, gcvi, evi])
print(combined_img.getInfo())

In [ ]:
# Define the path to the training data
training_data_path = "path/to/fiji-lulc-2021-test-data-modified.geojson"

# or for RLP:
#training_data_path = "path/to/traindata_rlp_modified.geojson"


# Convert the GeoJSON file to an Earth Engine object
ee_object = geemap.geojson_to_ee(training_data_path)
print(ee_object.getInfo())

In [26]:
# Sample the combined image at the locations of your training points
samples = combined_img.sampleRegions(
    collection=ee_object,
    properties=['Label'],  
    scale=10,
    geometries=True
)

In [ ]:
# Export the sample data to a CSV file in your Google Drive
task = ee.batch.Export.table.toDrive(
    collection=samples,
    description='Training_Samples_Export',
    fileFormat='CSV'
)

task.start()

# Monitor the task
import time
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)

print('Task completed!')

In [ ]:
# To visualize the map (optional)
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 2000,
    'gamma': 2
}

Map = geemap.Map(center=[-17.7134, 178.065], zoom=8)
Map.addLayer(combined_img, vis_params, "Sentinel-2 Indices")
Map.addLayer(samples, {}, "Training Points")
Map